<img width="8%" alt="GitHub.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/GitHub.png" style="border-radius: 15%">

# GitHub - Send contributor activity on slack
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Send+contributor+activity+on+slack:+Error+short+description">Bug report</a>

**Tags:** #github #activity #update #api #snippet #operations #slack 

**Author:** [Benjamin Filly](https://www.linkedin.com/in/benjamin-filly-05427727a/)

**Last update:** 2023-06-28 (Created: 2023-06-16)

**Description:** This notebook demonstrates how to send GitHub activity of a contributor of awesome notebook templates to Slack. It includes the sections below:
- ✅ **Templates created:** the total number of templates created (overall, by month, by week).
- 👀 **In review:** the number of PRs ready reviewed. Make sure you made a comment with **"Ready to review"** inside the PR.
- 🏗 **In progress:** the current PRs you are working on.
- 📋 **Backlog:** the issues you are assigned to with no PRs opened.

*NB: Execution time may takes between 2 to 5 min.*

**References:**
- [GitHub REST API Documentation](https://docs.github.com/en/rest/issues/issues?apiVersion=2022-11-28#update-an-issue)
- [os](https://docs.python.org/3/library/os.html)
- [json](https://docs.python.org/3/library/json.html)
- [datetime](https://docs.python.org/3/library/datetime.html)
- [pandas](https://pandas.pydata.org/docs/)
- [requests](https://docs.python-requests.org/en/latest/)
- [slack (naas_drivers)](https://pypi.org/project/naas-drivers/)
- [Slack Block Kit Builder](https://app.slack.com/block-kit-builder/)

## Input

### Import libraries

In [ ]:
import os
import json
from datetime import datetime
from github import Github
import naas
import pandas as pd
import requests
from naas_drivers import slack
import warnings
warnings.filterwarnings("ignore")
# The function "pd.set_option('display.max_colwidth', None)" is used in pandas, a data manipulation library in Python, to set the maximum width for displaying column contents in a tabular format to be unlimited, allowing the full content of each column to be displayed without truncation.
pd.set_option('display.max_colwidth', None)

### Setup Variables
- `github_token`: [GitHub token](https://help.github.com/en/github/authenticating-to-github/creating-a-personal-access-token-for-the-command-line)
- `repo_name`: name of the repository in two part: owner_name/repository_name
- `contributor_profile`: your GitHub username
- `cron`: cron params for naas scheduler
- `slack_bot_token`: [Slack Bot token](https://docs.celigo.com/hc/en-us/articles/7140655476507-How-to-create-an-app-and-retrieve-OAuth-token-in-Slack)
- `slack_channel`: The name of the channel you want to send the message

In [ ]:
# Inputs
github_token = naas.secret.get("GITHUB_TOKEN") or "YOUR_GITHUB_TOKEN"
repo_name = "jupyter-naas/awesome-notebooks"
contributor_profile = "GITHUB_USERNAME" #example: "FlorentLvr" 
cron = "30 9,12,17 * * 1-5"

# Outputs
slack_bot_token = naas.secret.get("SLACK_BOT_TOKEN") or "YOUR_SLACK_TOKEN"
slack_channel = "YOUR_CHANNEL_NAME" #example: "naas-templates-contributors"

## Model

### Connect to GitHub repo

In [ ]:
# Connect to the GitHub API using pygithub library
g = Github(github_token)

# Get the repository
repo = g.get_repo(repo_name)

### Get PRs closed
So here we're filtering out all closed PRs on the `contributor_profile`, then we retrieve the files added by each PR and classify them by month and week.

In [ ]:
# Get the closed PR
pull_requests = repo.get_pulls(state='closed', sort='updated', direction='desc')
print("✅ Pull Requests fetched:", pull_requests.totalCount)

# Iterate over the pull requests and filter by assignee
assigned_pull_requests = [pr for pr in pull_requests if pr.assignee and pr.assignee.login == contributor_profile]

# Print the closed PR count
print(f"📌Number of assigned closed PR: {len(assigned_pull_requests)}")

### Get templates added by PRs closed

In [ ]:
# Create an empty list to store the files added in closed pull requests
files_added_month = []
files_added_week = []
files_added = []

# Get the current date
current_date = datetime.now()

# Iterate over the assigned closed pull requests
for pr in assigned_pull_requests:    
    # Get files added
    files = pr.get_files()
    
    # Get the date the file was created
    file_created = pr.merged_at
    
    days_diff = None  # Initialize days_diff
    
    if file_created is not None:
        # Calculate the difference in days between the current date and file creation date
        days_diff = (current_date - file_created).days
        
        # Iterate over the files added in the pull request
        for file in files:
            if file.status == "added":
                file_path = file.filename
                file_url = file.raw_url

                # Prep data
                tmp = {
                    "file_path": file_path,
                    "file_url": file_url,
                    "file_created": pr.merged_at,
                    "pr_number": pr.number,
                    "pr_title": pr.title,
                    "pr_url": pr.url,
                }

                # Add the data to the respective lists based on the file creation date
                if days_diff is not None and days_diff <= 30:
                    files_added_month.append(tmp)
                if days_diff is not None and days_diff <= 7:
                    files_added_week.append(tmp)

                # Add the data to the files_added list
                files_added.append(tmp)

# Calculate the variation between the current month and the last month
current_month = current_date.month
last_month = current_month - 1 if current_month > 1 else 12
files_added_current_month = [file for file in files_added_month if file['file_created'].month == current_month]
files_added_last_month = [file for file in files_added_month if file['file_created'].month == last_month]
variation_month = len(files_added_current_month) - len(files_added_last_month)

# Calculate the variation between the current week and the last week
current_week = current_date.isocalendar()[1]
last_week = current_week - 1 if current_week > 1 else 52
files_added_current_week = [file for file in files_added_week if file['file_created'].isocalendar()[1] == current_week]
files_added_last_week = [file for file in files_added_week if file['file_created'].isocalendar()[1] == last_week]
variation_week = len(files_added_current_week) - len(files_added_last_week)

# Prepare the variation strings
variation_month_str = f"+{variation_month}" if variation_month > 0 else str(variation_month)
variation_week_str = f"+{variation_week}" if variation_week > 0 else str(variation_week)

# Print the number of files added and the variations
print(f"📈 Number of Files added: {len(files_added)}")
print(f"📁 Number of Files added this month: {len(files_added_current_month)} ({variation_month_str} vs last month)")
print(f"📂 Number of Files added this week: {len(files_added_current_week)} ({variation_week_str} vs last week)")

### Get PRs opened

In [ ]:
# Get the closed PR
pull_requests = repo.get_pulls(state="open")

# Print the closed PR
print("✅ Pull Requests fetched:", pull_requests.totalCount)
data = []
for index, pr in enumerate(pull_requests):
    # Init
    assignee_login = None
    assignee = pr.raw_data.get("assignee")
    if assignee:
        assignee_login = assignee.get("login")
    tmp = {
        "title": pr.title,
        "number": pr.number,
        "url": pr.raw_data.get("url"),
        "assignee": assignee_login,
        "created_at": pr.raw_data.get("created_at"),
        "updated_at": pr.raw_data.get("updated_at"),
    }
    data.append(tmp)

df_pr = pd.DataFrame(data)
df_pr.head(3)

### Get PRs opened assigned to contributor

In [ ]:
contributor_filtered_df = df_pr[df_pr['assignee'] == contributor_profile].reset_index(drop=True)
print("✅ PR Opened:", len(contributor_filtered_df))
contributor_filtered_df.head(1)

### Get PRs "In progress" and "To be reviewed"

We loop over each PR to see if the last message is "Ready to review", if so we put it in the finished category otherwise in work in progress. We also look to see if there is a link to a PR in the description, which allows us to sort the new issues more easily and reliably.

In [ ]:
# Init - Create empty list
prs_in_review = []
prs_work_in_progress = []
prs_linked_to_issues = []
prs_numbers = []

# Loop in df and last message on each PR
for row in contributor_filtered_df.itertuples():
    # Display PR number
    pr_number = row.number
    
    # Append prs numbers
    prs_numbers.append(str(pr_number))
    
    # Get PR object
    pr = next((pr for pr in pull_requests if pr.number == pr_number), None)
    
    # If PR exists then get comments
    if pr:
        # Get PR description
        pr_description = pr.body
        issue_number = None
        if pr_description:
            # Split description to get issue number
            split_result = pr_description.split("https://github.com/jupyter-naas/awesome-notebooks/issues/")

            # If list > 1 then issue number exists
            if len(split_result) > 1:
                # Get issue number
                issue_number = split_result[1]
        if issue_number:
            prs_linked_to_issues.append(issue_number)

        # Get comments from PR
        comments = pr.get_issue_comments()
        
        # Check if nb comments > 0 else PR status = WIP
        if comments.totalCount > 0:
            last_comment = comments[comments.totalCount-1].body.lower()
            
            # Check if "read to review" else PR status = WIP
            if "ready to review" in last_comment:
                prs_in_review.append(pr)
            else:
                prs_work_in_progress.append(pr)
        else:
            prs_work_in_progress.append(pr)
            
print("✅ Pull Requests in Review:", len(prs_in_review))
print("👨‍💻 Pull Requests Work in Progress:", len(prs_work_in_progress))
print("💻 Issues linked in PRs:", len(prs_linked_to_issues))

### Get new issues by contributor
Here we retrieve the issues and sort them on the `contributor_profile`, then we exclude issues linked to a PR.

In [ ]:
# Get the repository
repository = g.get_repo(f'{repo_name}')

# Get issues assigned to the contributor
issues = repository.get_issues(assignee=contributor_profile)

# Print number of issues
print("✅ Issues fetched:", issues.totalCount)

# Init variables
new_issues = []

# Loop
for issue in issues:
    # Init
    issue_number = issue.number
    
    # List events in issues
    events = issue.get_events()
    list_events = [event.event for event in events]
    
    # Exclude issues linked to PR or with events connected
    if (str(issue_number) not in prs_linked_to_issues) and ('connected' not in list_events) and (str(issue_number) not in prs_numbers):
        new_issues.append(issue)

print("✅ New issues:", len(new_issues))

### Create Slack message with blocks
We used [Block kit builder](https://app.slack.com/block-kit-builder/) to create our templates and adapt it with the dynamic variables.

In [ ]:
# Sort the lists
prs_in_review.sort(key=lambda pr: pr.number, reverse=True)
prs_work_in_progress.sort(key=lambda pr: pr.number, reverse=True)
new_issues.sort(key=lambda issue: issue.number, reverse=True)

# Get the current date and time
current_date = datetime.now().strftime("%B %d, %Y %H:%M")

# Calculate the variations
variation_month_str = f"+{variation_month}" if variation_month > 0 else str(variation_month)
variation_week_str = f"+{variation_week}" if variation_week > 0 else str(variation_week)

# Blocks
blocks = [
    {
        "type": "header",
        "text": {
            "type": "plain_text",
            "text": f"{contributor_profile} - Activity update as of {current_date}",
            "emoji": True
        }
    },
    {
        "type": "divider"
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": f"✅ *Templates created: {len(files_added)}*",
        }
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": f"This month: {len(files_added_current_month)} ({variation_month_str} vs last month)\nThis week: {len(files_added_current_week)} ({variation_week_str} vs last week)"
        }
    },
    {
        "type": "divider"
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": f"👀 *In review: {len(prs_in_review)}*"
        }
    },
    *[
        {
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"- <{pr.html_url}|#{pr.number} {pr.title}>"
            }
        }
        for pr in prs_in_review
    ],
    {
        "type": "divider"
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": f"🏗 *In progress: {len(prs_work_in_progress)}*"
        }
    },
    *[
        {
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"- <{pr.html_url}|#{pr.number} {pr.title}>"
            }
        }
        for pr in prs_work_in_progress
    ],
    {
        "type": "divider"
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": f"📋 *Backlog: {len(new_issues)}*"
        }
    },
    *[
        {
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"- <{issue.html_url}|#{issue.number} {issue.title}>"
            }
        }
        for issue in new_issues
    ]
]

## Output

### Send message on Slack

In [ ]:
slack.connect(slack_bot_token).send(slack_channel, text=None, blocks=blocks)

### Add scheduler

In [ ]:
naas.scheduler.add(cron=cron)